In [16]:
# read jsonl file and display it in csv format through pandas
import pandas as pd
import json
import csv
import sys
import ast
import os
from PIL import Image as PILImage
from datasets import Dataset, Features, Value, Image

file = "llm_scene_description_with_image.jsonl"

def read_jsonl(file):
	with open(file, 'r') as f:
		data = f.readlines()
	return data

def jsonl_to_csv(data):
	with open('llm_scene_description_with_image.csv', 'w', newline='') as f:
		writer = csv.writer(f)
		header_written = False
		for line in data:
			item = json.loads(line)
			if not header_written:
				writer.writerow(item.keys())
				header_written = True
			writer.writerow(item.values())
   
data = read_jsonl(file)
jsonl_to_csv(data)
df = pd.read_csv('llm_scene_description_with_image.csv')


def process_gen_details(x):
	try:
		# Convert string representation of dict to actual dict
		d = ast.literal_eval(x)
		# Convert dict to a plain text string representation
		return ", ".join([f"{k}: {v}" for k, v in d.items()])
	except Exception as e:
		return x

df['generation_details'] = df['generation_details'].apply(process_gen_details)

# Improved helper function to load an image and convert it to RGB

def load_image(path):
    if not os.path.exists(path):
        print(f"File not found: {path}")
        return None
    try:
        with PILImage.open(path) as img:
            return img.convert("RGB")
    except Exception as error:
        print(f"Error loading {path}: {error}")
        return None


# Create a Hugging Face Dataset from the edited dataframe, letting the Image feature load images from the file paths
dataset = Dataset.from_pandas(
    df,
    features=Features(
        {
            "prompt_model": Value("string"),
            "prompt": Value("string"),
            "task_type": Value("string"),
            "diffusion_model": Value("string"),
            "image_path": Image(),  # Use the image_path column as the image feature
            "generation_details": Value("string"),
        }
    ),
)
dataset


Dataset({
    features: ['prompt_model', 'prompt', 'task_type', 'diffusion_model', 'image_path', 'generation_details'],
    num_rows: 10
})

In [26]:

dataset.push_to_hub("test_dataset", private=True)  # Replace 'your-username' with your actual Hugging Face username.

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/stogian/test_dataset/commit/a803433d97210f446a53af892d7fb86760447005', commit_message='Upload dataset', commit_description='', oid='a803433d97210f446a53af892d7fb86760447005', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/stogian/test_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='stogian/test_dataset'), pr_revision=None, pr_num=None)

In [7]:
from datasets import Dataset, Image, Features, Value
import json
from PIL import Image as PILImage
import os
import glob


def generate_examples():
	for data in annotations:
		image_path = os.path.join("Spatial-MM/data", data["image_name"])
		try:
			image = PILImage.open(image_path).convert("RGB")  # Load image as RGB
		except Exception as e:
			print("Error loading image: ", image_path)
			continue
		yield {
			"image": image,
			"question": data["question"],
			"answer": data["answer"],
		}


	# Load JSON annotations from multiple files
annotations = []
json_files = glob.glob("Spatial-MM/data/Spatial_MM_CoT/*.json")
for json_file in json_files:
	with open(json_file, "r") as f:
		annotations.extend(json.load(f))

print(f"Loaded {len(annotations)} annotations")

# Define dataset features (adjust based on your JSON structure)
# features = Features(
# 	{
# 		"image": Image(),
# 		"question": Value("string"),
# 		"answer": Value("string"),
# 	}
# )

# # Create the dataset
# dataset = Dataset.from_generator(
# 	generate_examples,
# 	features=features,
# )

# dataset
	# dataset.push_to_hub("spatial_mm", private=True)  # Push to the Hub



Loaded 2211 annotations


In [ ]:
import shutil
import os

source_dirs = ['Spatial-MM/data/Spatial_MM_Obj', 'Spatial-MM/data/Spatial_MM_CoT']
target_dir = 'Spatial-MM/data/images'

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

for source in source_dirs:
	for filename in os.listdir(source):
		source_file = os.path.join(source, filename)
		if os.path.isfile(source_file):
			shutil.move(source_file, target_dir)

In [ ]:
image_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]

source_dirs = ["Spatial-MM/data/Spatial_MM_Obj", "Spatial-MM/data/Spatial_MM_CoT"]
target_dir = "Spatial-MM/data/image_dir"

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

for source in source_dirs:
	for filename in os.listdir(source):
		if os.path.splitext(filename)[1].lower() in image_extensions:
			shutil.move(os.path.join(source, filename), target_dir)


In [10]:
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
image_files = [
	f for f in os.listdir('Spatial-MM/data/Spatial_MM_Obj')
	if os.path.splitext(f)[1].lower() in image_extensions
]
print(f"Number of images: {len(image_files)}")

Number of images: 5702


In [1]:
from datasets import Dataset, Image, Features, Value
import json
from PIL import Image as PILImage
import os
import glob


def generate_examples():
	for data in annotations:
		image_path = os.path.join("data/egobrientbench", data["image"])
		try:
			image = PILImage.open(image_path).convert("RGB")  # Load image as RGB
		except Exception as e:
			print("Error loading image: ", image_path)
			continue
		yield {
			"image": image,
			"question": data["question"],
			"answer": data["label"],
		}

	# Load JSON annotations from multiple files


annotations = []
json_files = glob.glob("data/egobrientbench/benchmark.json")
for json_file in json_files:
	with open(json_file, "r") as f:
		annotations.extend(json.load(f))

print(f"Loaded {len(annotations)} annotations")

# Define dataset features (adjust based on your JSON structure)
features = Features(
	{
		"image": Image(),
		"question": Value("string"),
		"answer": Value("string"),
	}
)

# Create the dataset
dataset = Dataset.from_generator(
	generate_examples,
	features=features,
)

dataset
# dataset.push_to_hub("spatial_mm", private=True)  # Push to the Hub


Loaded 33460 annotations


Dataset({
    features: ['image', 'question', 'answer'],
    num_rows: 33460
})

In [3]:
dataset.push_to_hub("stogian/egoorientbench", private=True )  # Push to the Hub

Uploading the dataset shards:   0%|          | 0/9 [00:00<?, ?it/s]

Map:   0%|          | 0/3718 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

Map:   0%|          | 0/3718 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/3717 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

Map:   0%|          | 0/3717 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/stogian/egoorientbench/commit/555bae171477e4755b0193eed2cc9f6c42b4d6e5', commit_message='Upload dataset', commit_description='', oid='555bae171477e4755b0193eed2cc9f6c42b4d6e5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/stogian/egoorientbench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='stogian/egoorientbench'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import notebook_login

!pip install transformers


model_name = "bert-base-uncased"
model = AutoModel.from_pretrained(model_name, cache_dir="../bin/models/"+model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="../bin/models/"+model_name)
notebook_login()